# Single Track Kinemematic Study

This code experiments with the single track kinematic model commonly used in autonomous vehicle research.

In [4]:
import matplotlib as plt
import numpy as np
import scipy.integrate as grate
from math import cos, sin, pi

Assume there is a 2D intertial frame ($\mathbb{R}^2$) with basis vectors $e_x$ and $e_y$.
The single track kinematic model assumes two non-slip wheels with center points that are co-linear along a rigid link with length $l$.

There are two important angles in the model:

1. $\theta$ - the angle between $p_f - p_r$ and $e_x$
2. $\delta$ - the rotation angle between the front wheel's forward velocity, $\dot{p}_f$, and $p_f - p_r$.

Let's consider the model for the front wheel:

$\dot{x}_f = v_f cos(\theta + \delta)$

$\dot{y}_f = v_f cos(\theta + \delta)$

$\dot{\theta} = \frac{v_f}{l}\sin{\delta}$


The front and rear wheel speeds are related by:
$$
\frac{v_r}{v_f}=\cos{\theta}
$$

The control problem involves choosing a forward speed, $v_f \in {[v_{min},v_{max}]}$, and some steering angle, $\delta \in {[\delta_{min},\delta_{mx}]}$.

Time to throw some numerical tools at this model...

In [14]:
def single_track_front(l, t, p, u):
    '''
    Parameters:
    l (float) - length of rigid link between front and rear wheel
    p (vector[float]) - vehicle configuration, [x, y, theta]
    u (vector[float]) - control inputs, vf and delta
    '''
    theta = p[2]
    vf = u[0]
    delta = u[1]
    x_dot = vf * cos(theta + delta)
    y_dot = vf * sin(theta + delta)
    theta_dot = (vf / l)*sin(delta)
    return [x_dot, y_dot, theta_dot]

In [15]:
# Test the function out to make sure it works with reasonable input.
single_track_front(1.0, 0.0, [0.0,0.0,0.0], [1.0, 0.1])

[0.9950041652780257, 0.09983341664682815, 0.09983341664682815]

In [16]:
from functools import partial

In [ ]:
# Simulate this system for 5 seconds with a constant steering angle
tf = 5
tspan = (0,tf)
# Initial configuration, p = [x,y,theta]
p0 = [0.0,0.0,0.0]

